In [11]:
import numpy as np
import cv2
import math
import time
import collections

In [12]:
roi_count = 0
last_roi = None
state = True
state_count = 10
frame_no = 0
gesture_list = []

In [13]:
hand_cascade = cv2.CascadeClassifier('aGest.xml')
cap = cv2.VideoCapture(0)

In [14]:
def getROI(frame, hand_cascade):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rois = cascade.detectMultiScale(gray, 1.3, 5)
    if len(rois) > 0:
        return rois[0]
    
    return None

In [18]:
def detectGesture(src, roi_rect, thresh):
    global gesture_list
    p, q, r, s = roi_rect
    image, contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    line_count = 0;
    
    if len(countours):
        cnt = max(contours, key = lambda x: cv2.contourArea(x))
    
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(src, (p+x, p+y), (p+x+w, q+y+h), (0, 0, 255), 0)
        
        hull = cv2.convexHull(cnt)
        
        drawing = np.zeros(src.shape, np.uint8)
        
        hull = cv2.convexHull(cnt, returnPoints = False)
        
        defects = cv2.convexityDefects(cnt, hull)
        count_defects = 0
        
        #cosine rule
        
        if (defects is not None and len(defects) > 0):
            if(defects.shape[0] != None):
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i, 0]
                    start = tuple(cnt[s][0])
                    end = tuple(cnt[e][0])
                    far = tuple(cnt[f][0])
                    
                    ls = [[start[0], start[1]], [end[0], end[1]], [far[0], far[1]]]
                    ctr = np.array(ls).reshape((-1, 1, 2)).astype(np.int32)
                    
                    a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b= math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    
                    #cosine rule
                    angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
                    
                    if angle < 90 and cv2.contourArea(ctr) > 2000:
                        count_defects += 1
                        far = (far[0] + p, far[1] + q)
                        cv2.circle(src, far, 1, [0, 0, 255], -1)
                        
                        
                    start = (start[0] + p, start[1] + q)
                    end = (end[0] + p, end[1] + q)
                    
                    cv2.line(src, start, end, [0, 255, 0], 2)
                    
                gesture_list.append(count_defects+1)
                
    cv2.imshow('frame', src)
                        

In [15]:
def gestureRecognition(src, roi_rect, hand_roi):
    lower = np.array([0, 48, 80], dtype = "uint8")
    upper = np.array([20, 255, 255], dtype = "uint8")
    
    converted = cv2.cvtColor(hand_roi, cv2.COLOR_BGR2HSV)
    skinMask = cv2.inRange(converted, lower, upper)
    
    kernel = np.ones((5,5), np.uint8)
    
    img_dilated = cv2.dilate(skinMask, kernel, iterations = 3)
    
    detectGesture(src, roi_rect, img_dilated)

In [ ]:
while True:
    ret, frame = cap.read() #capture frame by frame
    
    roi = getROI(frame, hand_cascade) #get the ROI of palm ****
    
    if not(roi is None): #10 consecutive palm detections
        if roi_count != 10:
            roi_count += 1
            
        if roi_count == 10:
            cv2.putText(frame, "Event Detected", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 1, 2)
        
        last_roi = roi
        x, y, w, h = roi
        frame = cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 2)
        state = True
        
    else:
        
        if (not state and state_count > 0):
            #update boundary
            x, y, w, h = last_roi
            x = x - w
            y = y - h
            w = w * 3
            h = h * 2
            if x < 0:
                x = 0
            if y < 0:
                y = 0
                
            state_count = state_count - 1
            
            roi_rect = x, y, w, h
            
            #hand gesture
            
            gestureRecognition(frame, roi_rect, frame[y:y+h, x:x+h])
            
        if (not state and state_count == 0):
            

In [20]:
cap.release()